In [2]:
import numpy as np
import sys

sys.setrecursionlimit(1_000_000)

with open('input.txt') as f:
    lines = f.readlines()

width = len(lines[0].strip())
height = len(lines)

island = np.ones((height + 2, width + 2))

legend = {
    '#': 1,
    '.': 0,
    '^': 2,
    '>': 3,
    'v': 4,
    '<': 5,
}

legend2 = {
    -1: '*',
    1: '-',
    0: '.',
    2: '^',
    3: '>',
    4: 'v',
    5: '<',
}

road_fields = []

start = None
end = None

for y, line in enumerate(lines):
    for x, symbol in enumerate(line.strip()):
        island[y + 1, x + 1] = legend[symbol]
        if island[y + 1, x + 1] == 0 and start is None:
            start = (y + 1, x + 1)
        if island[y + 1, x + 1] != 1:
            road_fields.append((y + 1, x + 1))
        if y == len(lines) - 1 and symbol == '.':
            end = (y + 1, x + 1)

# print('start', start, 'end', end, 'fields', len(road_fields))


# print(island)


class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.adjacency_list = {vertex: [] for vertex in vertices}

    def add_edge(self, u, v):
        self.adjacency_list[u].append(v)
        # self.adjacency_list[v].append(u)


def get_neighbors(pos):
    y, x = pos
    radar = island[max(0, y - 1):min(island.shape[0], y + 2), max(0, x - 1):min(island.shape[1], x + 2)]
    return radar


g = Graph(road_fields)

count = 0
TOP = (1, 0)
BOTTOM = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)


def build_graph(current, visited):
    queue = [current]
    
    while len(queue) > 0:
        current = queue.pop(0)
        if current in visited:
            continue
        y, x = current
        visited.append(current)
        top = (y - 1, x)
        bottom = (y + 1, x)
        left = (y, x - 1)
        right = (y, x + 1)
        if island[left] != 1:
            g.add_edge(current, left)
            queue.append(left)
        if island[top] != 1:
            g.add_edge(current, top)
            queue.append(top)
        if island[right] != 1:
            g.add_edge(current, right)
            queue.append(right)
        if island[bottom] != 1:
            g.add_edge(current, bottom)
            queue.append(bottom)


build_graph(start, [])

max_path = None
fend = 0

def dfs(graph, start, visited, max_distance, farthest_vertex, mypath):
    global fend, max_path
    fend += 1
    # print('start', start)
    if start == end:
        print('end', fend, max_distance, farthest_vertex)
        if max_path is None or len(mypath) > len(max_path):
            max_path = [p for p in mypath]
        return max_distance
    visited.append(start)
    mypath.append(start)

    y, x = start
    curr_land = island[start]
    dist = 0

    if curr_land in [2, 3, 4, 5]:
        if curr_land == 2:
            dist = dfs(graph, (y - 1, x), visited, max_distance + 1, farthest_vertex, mypath)
        elif curr_land == 3:
            dist = dfs(graph, (y, x + 1), visited, max_distance + 1, farthest_vertex, mypath)
        elif curr_land == 4:
            dist = dfs(graph, (y + 1, x), visited, max_distance + 1, farthest_vertex, mypath)
        else:
            dist = dfs(graph, (y, x - 1), visited, max_distance + 1, farthest_vertex, mypath)
    else:
        for neighbor in graph.adjacency_list[start]:
            # '^': 2,
            # '>': 3,
            # 'v': 4,
            # '<': 5,
            y1, x1 = neighbor
            land = island[neighbor]
            if land == 5 and x < x1:
                continue
            if land == 3 and x > x1:
                continue
            if land == 2 and y < y1:
                continue
            if land == 4 and y > y1:
                continue
            if neighbor not in visited:
                dist = dfs(graph, neighbor, visited, max_distance + 1, farthest_vertex, mypath)
                if neighbor in visited:
                    visited.remove(neighbor)
                    
    # print('max_distance', max_distance)
    if dist > max_distance: # > farthest_vertex[0]:
        farthest_vertex[0] = dist # max_distance
        farthest_vertex[1] = start
    
    mypath.remove(start)

    return dist


def find_diameter(graph, start):
    visited = []
    mypath = []

    farthest_vertex = [0, start]
    dfs(graph, start, visited, 0, farthest_vertex, mypath)
    # print(len(visited), visited)
    
    return farthest_vertex[0]


# diameter = find_diameter(g, start)
# # print("Диаметр дерева:", diameter)
# 
# # print(paths)
# 
# print(f'\n------ {len(max_path)} ------')
# l = []
# for y, row in enumerate(island):
#     s = []
#     for x, col in enumerate(row):
#         if (y, x) in max_path:
#             s.append('*')
#         else:
#             s.append(legend2[col])
#     l.append(''.join(s))
# 
# print('\n'.join(l))


In [ ]:
from functools import lru_cache

max_path_2 = None

def dfs_2(graph, start, visited, mypath):
    global max_path_2
    queue = [start]
    
    while queue:
        start = queue.pop()
        if start in visited:
            print('???', start)
            print(f'\n------ {len(visited)} ------')
            l = []
            for y, row in enumerate(island):
                s = []
                for x, col in enumerate(row):
                    if (y, x) in mypath:
                        s.append('*')
                    else:
                        s.append(legend2[col])
                l.append(''.join(s))
            
            print('\n'.join(l))
            continue
        
        if start in visited:
            print('???', start)
        visited.append(start)
        if start in mypath:
            print('!!!', start)
        mypath.append(start)
        # if callable(start):
        #     # try:
        #     start()
        #     print('success')
        #     # except Exception:
        #     #     pass
        #     continue
            
        if start == end:
            print('end', len(mypath))
            if max_path_2 is None or len(mypath) > len(max_path_2):
                max_path_2 = [p for p in mypath]
            continue

        queue.extend(neighbor for neighbor in graph.adjacency_list[start] if neighbor not in visited)
            
        # mypath.append(start)
        # 
        # queue.append(lambda : mypath.remove(start))
        # 
        # for neighbor in graph.adjacency_list[start]:
        #     if neighbor not in visited:
        #         queue.append(lambda : print('visited.remove', start) and visited.remove(neighbor))
        #         queue.append(neighbor)


visited = set()
mypath = []
graph = g

max_dist = 0

def dfs_3(start, dist):
    global max_path_2, max_dist
    if start == end:
        if dist > max_dist:
            max_dist = dist
            print('max dist', max_dist)
        # if max_path_2 is None or len(mypath) > len(max_path_2):
        #     print(len(mypath))
        #     max_path_2 = [p for p in mypath]
        return
    visited.add(start)
    # mypath.append(start)

    for neighbor in graph.adjacency_list[start]:
        if neighbor not in visited:
            dfs_3(neighbor, dist + 1)
            # if neighbor in visited:

    visited.remove(start)
    # mypath.remove(start)


dfs_3(start, 0)

print('max_dist', max_dist)

# print(f'\n------ {len(max_path_2)} ------')
# l = []
# for y, row in enumerate(island):
#     s = []
#     for x, col in enumerate(row):
#         if (y, x) in max_path_2:
#             s.append('*')
#         else:
#             s.append(legend2[col])
#     l.append(''.join(s))
# 
# print('\n'.join(l))



max dist 4726
max dist 4994
max dist 5050
max dist 5218
max dist 5350
max dist 5618
max dist 5674
max dist 5750
max dist 5806
max dist 5834
max dist 5866
max dist 5922
max dist 5950
max dist 5986
max dist 6026
max dist 6042
max dist 6062
max dist 6094
max dist 6214
max dist 6254
max dist 6290
max dist 6406
max dist 6430
max dist 6470
max dist 6490
max dist 6526
max dist 6566
max dist 6570
max dist 6578
max dist 6602
max dist 6642
max dist 6662
max dist 6698
max dist 6738


In [5]:
15+22+10+10+38+10+5

110